# Convert Connect4 Models from .keras to .h5

Run this in Google Colab. Connect your Drive, set the model folder path below, then run all cells.

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Models from Connect4_CNN_Transformer_Training.ipynb
# CNN:        connect4_cnn_best.keras, connect4_cnn_final.keras
# Transformer: connect4_transformer_best.keras, connect4_transformer_final.keras
MODEL_FOLDER = "/content/drive/MyDrive/Connect4_Combined/models"

In [4]:
import os

# List files in the folder to confirm path
if os.path.exists(MODEL_FOLDER):
    print(f"Files in {MODEL_FOLDER}:")
    for f in os.listdir(MODEL_FOLDER):
        print(f"  - {f}")
else:
    print(f"Folder not found: {MODEL_FOLDER}")
    print("Edit MODEL_FOLDER in the cell above.")

Files in /content/drive/MyDrive/Connect4_Combined/models:
  - connect4_cnn_best.keras
  - connect4_cnn_final.keras
  - connect4_transformer_best.keras
  - connect4_transformer_final.keras
  - cnn_vs_transformer.png


In [5]:
!pip install -q "tensorflow>=2.16"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 909.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 73.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 93.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 19.5 MB/s eta 0:00:00


In [7]:
# Custom layers from Connect4_CNN_Transformer_Training.ipynb (required to load Transformer)
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class BoardPatchEmbedding(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        self.proj = layers.Dense(self.embed_dim)
        super().build(input_shape)

    def call(self, x):
        x = tf.reshape(x, [-1, 42, 2])
        return self.proj(x)

class SinusoidalPositionalEmbedding(layers.Layer):
    def __init__(self, seq_len, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.seq_len = seq_len
        self.embed_dim = embed_dim

    def build(self, input_shape):
        pe = np.zeros((1, self.seq_len, self.embed_dim), dtype=np.float32)
        for pos in range(self.seq_len):
            for i in range(0, self.embed_dim, 2):
                pe[0, pos, i] = np.sin(pos / 10000**(i / self.embed_dim))
                if i + 1 < self.embed_dim:
                    pe[0, pos, i+1] = np.cos(pos / 10000**(i / self.embed_dim))
        self.pos_emb = self.add_weight(name='pos_emb', shape=(1, self.seq_len, self.embed_dim),
                                       initializer=keras.initializers.Constant(pe), trainable=False)
        super().build(input_shape)

    def call(self, x):
        return x + self.pos_emb

CUSTOM_OBJECTS = {
    'BoardPatchEmbedding': BoardPatchEmbedding,
    'SinusoidalPositionalEmbedding': SinusoidalPositionalEmbedding,
}

In [8]:
import tensorflow as tf
import os

for name in ["connect4_cnn_final", "connect4_transformer_final"]:
    keras_path = os.path.join(MODEL_FOLDER, f"{name}.keras")
    h5_path = os.path.join(MODEL_FOLDER, f"{name}.h5")
    
    if not os.path.exists(keras_path):
        print(f"Skipping {name}: {keras_path} not found")
        continue
    
    print(f"Converting {name}.keras -> {name}.h5 ...")
    # Transformer needs custom_objects for BoardPatchEmbedding & SinusoidalPositionalEmbedding
    custom = CUSTOM_OBJECTS if "transformer" in name else {}
    model = tf.keras.models.load_model(keras_path, custom_objects=custom)
    model.save(h5_path)
    size_mb = os.path.getsize(h5_path) / (1024 * 1024)
    print(f"  Done: {h5_path} ({size_mb:.1f} MB)")

print("\nAll done. Download the .h5 files from Drive or copy to your project.")

Converting connect4_cnn_final.keras -> connect4_cnn_final.h5 ...
  Done: /content/drive/MyDrive/Connect4_Combined/models/connect4_cnn_final.h5 (5.3 MB)
Converting connect4_transformer_final.keras -> connect4_transformer_final.h5 ...


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 270 variables whereas the saved optimizer has 274 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


  Done: /content/drive/MyDrive/Connect4_Combined/models/connect4_transformer_final.h5 (49.5 MB)

All done. Download the .h5 files from Drive or copy to your project.
